# Portfolio Collections Dashboard

**Goal:** Visualise arrears and default trends to support proactive collections management for a water utility company.

This analysis reuses the same dataset as the Predicting Customer Payment Default Risk project to create actionable insights for collections teams.

## 1) Setup & Data Loading

In [ ]:
# Imports & Paths
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

# Database connection
DB_PATH = '../data/water_collections_demo.sqlite'
con = sqlite3.connect(DB_PATH)

print(f"Connected to database: {DB_PATH}")
print(f"Working directory: {os.getcwd()}")
print("📊 Interactive dashboard components loaded!")

In [ ]:
# Sanity Checks - Verify data loaded correctly
tables = ['customers', 'bills', 'payments', 'collections_actions']

print("Data verification:")
for table in tables:
    count = pd.read_sql(f'SELECT COUNT(*) as n FROM {table}', con).iloc[0, 0]
    print(f"{table:20}: {count:,} rows")

# Display sample data from each table
print("\n=== SAMPLE DATA FROM EACH TABLE ===")
for table in tables:
    print(f"\n{table.upper()} (First 3 rows):")
    sample_data = pd.read_sql(f'SELECT * FROM {table} LIMIT 3', con)
    display(sample_data)

## 2) SQL Views & Helper Indexes

Database indexes and analytical views have been pre-created for optimal performance.

In [ ]:
# Verify indexes exist
indexes = pd.read_sql("SELECT name FROM sqlite_master WHERE type='index' AND name LIKE 'ix_%'", con)
print(f"Custom indexes created: {len(indexes)}")
for idx in indexes['name']:
    print(f"  ✓ {idx}")

In [ ]:
# Verify bill_targets view exists and test it
view_test = pd.read_sql("""
SELECT 
    COUNT(*) as total_bills,
    SUM(default_60d) as defaults,
    ROUND(AVG(default_60d), 4) as default_rate,
    ROUND(AVG(bill_amount), 2) as avg_bill,
    ROUND(AVG(paid_in_window), 2) as avg_paid
FROM bill_targets
""", con)

print("KPI View Validation:")
display(view_test)

# Show sample of bill_targets view
print("\nSample from bill_targets view:")
sample_targets = pd.read_sql("SELECT * FROM bill_targets LIMIT 5", con)
display(sample_targets)

## 3) KPI Definitions (Arrears, Default Rate, Arrangements)

**Default Definition**: A bill is considered in default if it's not paid within the D-3 to D+60 window (3 days before due date to 60 days after due date).

**Payment Matching**: Payments are matched to bills with a £1 tolerance to account for rounding differences.

**Key Metrics**:
- **Default Rate**: Percentage of bills that default within the 60-day collection window
- **Monthly Trends**: Default rates aggregated by billing month
- **Segmentation**: Default rates by customer demographics (income, region)
- **Collections Activity**: Volume and distribution of collection actions

## 4) Portfolio EDA & Data Tables

In [ ]:
# Monthly trend analysis
kpi_monthly = pd.read_sql("""
SELECT 
    strftime('%Y-%m', bill_period_end) AS month,
    AVG(default_60d) AS default_rate,
    SUM(bill_amount) AS billed,
    SUM(paid_in_window) AS paid,
    COUNT(*) as bill_count
FROM bill_targets 
GROUP BY 1 
ORDER BY 1
""", con)

print(f"Monthly data: {len(kpi_monthly)} months")
print("\n📊 MONTHLY KPI DATA (All months):")
display(kpi_monthly)

In [ ]:
# Income band segmentation
seg_income = pd.read_sql("""
SELECT 
    c.income_band,
    ROUND(AVG(bt.default_60d), 3) AS default_rate,
    ROUND(AVG(bt.bill_amount), 2) AS avg_bill,
    COUNT(*) AS n_bills,
    COUNT(DISTINCT bt.customer_id) AS n_customers
FROM bill_targets bt 
JOIN customers c USING(customer_id) 
GROUP BY c.income_band 
ORDER BY default_rate DESC
""", con)

print("💰 INCOME BAND SEGMENTATION:")
display(seg_income)

In [ ]:
# Regional segmentation
seg_region = pd.read_sql("""
SELECT 
    c.region,
    ROUND(AVG(bt.default_60d), 3) AS default_rate,
    COUNT(*) AS n_bills,
    COUNT(DISTINCT bt.customer_id) AS n_customers,
    ROUND(AVG(bt.bill_amount), 2) AS avg_bill
FROM bill_targets bt 
JOIN customers c USING(customer_id) 
GROUP BY c.region 
ORDER BY default_rate DESC
""", con)

print("🌍 REGIONAL SEGMENTATION:")
display(seg_region)

In [ ]:
# Collections actions distribution
actions_dist = pd.read_sql("""
SELECT 
    action,
    COUNT(*) as n,
    COUNT(DISTINCT customer_id) as unique_customers,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM collections_actions), 2) as percentage
FROM collections_actions 
GROUP BY action 
ORDER BY n DESC
""", con)

print("📞 COLLECTIONS ACTIONS DISTRIBUTION:")
display(actions_dist)

In [ ]:
# Additional analysis tables
print("\n=== ADDITIONAL ANALYSIS TABLES ===")

# Tariff analysis
tariff_analysis = pd.read_sql("""
SELECT 
    c.tariff,
    COUNT(DISTINCT c.customer_id) as customers,
    ROUND(AVG(bt.default_60d), 3) AS default_rate,
    ROUND(AVG(bt.bill_amount), 2) AS avg_bill
FROM customers c
JOIN bill_targets bt USING(customer_id)
GROUP BY c.tariff
ORDER BY default_rate DESC
""", con)

print("\n⚡ TARIFF ANALYSIS:")
display(tariff_analysis)

# Vulnerability analysis
vulnerability_analysis = pd.read_sql("""
SELECT 
    CASE WHEN c.is_vulnerable = 1 THEN 'Vulnerable' ELSE 'Not Vulnerable' END as vulnerability_status,
    COUNT(DISTINCT c.customer_id) as customers,
    ROUND(AVG(bt.default_60d), 3) AS default_rate,
    ROUND(AVG(bt.bill_amount), 2) AS avg_bill
FROM customers c
JOIN bill_targets bt USING(customer_id)
GROUP BY c.is_vulnerable
ORDER BY default_rate DESC
""", con)

print("\n🛡️ VULNERABILITY ANALYSIS:")
display(vulnerability_analysis)

## 5) Interactive Dashboard

In [ ]:
# Simple Interactive Dashboard that actually works
print("🚀 INTERACTIVE DASHBOARD")
print("Use the controls below to explore the data interactively.")

# Create interactive widgets
def create_working_dashboard():
    # Date range selector
    date_options = kpi_monthly['month'].tolist()
    start_date = widgets.Dropdown(
        options=date_options,
        value=date_options[0],
        description='Start Date:'
    )
    
    end_date = widgets.Dropdown(
        options=date_options,
        value=date_options[-1],
        description='End Date:'
    )
    
    # Metric selector
    metric_selector = widgets.Dropdown(
        options=[('Default Rate', 'default_rate'), ('Billed Amount', 'billed'), ('Paid Amount', 'paid')],
        value='default_rate',
        description='Metric:'
    )
    
    # Segment selector
    segment_selector = widgets.Dropdown(
        options=[('Income Band', 'income'), ('Region', 'region'), ('Tariff', 'tariff')],
        value='income',
        description='Segment:'
    )
    
    def update_dashboard(start, end, metric, segment):
        # Filter data
        start_idx = date_options.index(start)
        end_idx = date_options.index(end)
        
        if start_idx > end_idx:
            print("❌ Start date must be before end date")
            return
        
        filtered_data = kpi_monthly.iloc[start_idx:end_idx+1]
        
        # Create trend chart
        fig1 = px.line(
            filtered_data,
            x='month',
            y=metric,
            title=f'📈 {metric.replace("_", " ").title()} Trend ({start} to {end})',
            markers=True,
            height=400
        )
        fig1.update_layout(title_x=0.5)
        fig1.show()
        
        # Create segmentation chart
        if segment == 'income':
            seg_data = seg_income
            x_col = 'income_band'
        elif segment == 'region':
            seg_data = seg_region
            x_col = 'region'
        else:  # tariff
            seg_data = tariff_analysis
            x_col = 'tariff'
        
        fig2 = px.bar(
            seg_data,
            x=x_col,
            y='default_rate',
            title=f'🎯 Default Rate by {segment.title()}',
            color='default_rate',
            color_continuous_scale='Viridis',
            height=400
        )
        fig2.update_layout(title_x=0.5)
        if segment == 'region':
            fig2.update_layout(xaxis_tickangle=-45)
        fig2.show()
        
        # Show summary stats
        avg_metric = filtered_data[metric].mean()
        print(f"\n📊 SUMMARY STATISTICS:")
        print(f"   Period: {start} to {end}")
        print(f"   Average {metric.replace('_', ' ').title()}: {avg_metric:,.2f}")
        print(f"   Data points: {len(filtered_data)}")
        
        # Show filtered data table
        print(f"\n📋 FILTERED DATA TABLE:")
        display(filtered_data)
    
    # Create the interactive widget
    interactive_widget = widgets.interactive(
        update_dashboard,
        start=start_date,
        end=end_date,
        metric=metric_selector,
        segment=segment_selector
    )
    
    # Display the dashboard
    display(widgets.VBox([
        widgets.HTML("<h3>🎛️ Dashboard Controls</h3>"),
        widgets.HBox([start_date, end_date]),
        widgets.HBox([metric_selector, segment_selector]),
        interactive_widget.children[-1]
    ]))

create_working_dashboard()

## 6) Enhanced Interactive Charts

In [ ]:
# Enhanced Interactive Charts
print("🎨 ENHANCED INTERACTIVE VISUALISATIONS")
print("These charts provide hover details, zoom, and pan functionality.")

# 1. Monthly trend with hover details
fig_monthly = px.line(
    kpi_monthly,
    x='month',
    y='default_rate',
    title='📈 Monthly Default Rate Trend (Interactive)',
    markers=True,
    hover_data=['billed', 'paid', 'bill_count']
)
fig_monthly.update_layout(height=500, title_x=0.5, hovermode='x unified')
fig_monthly.show()

# 2. Income band analysis with detailed hover
fig_income = px.bar(
    seg_income,
    x='income_band',
    y='default_rate',
    title='💰 Default Rate by Income Band (Interactive)',
    color='default_rate',
    color_continuous_scale='Viridis',
    hover_data=['avg_bill', 'n_bills', 'n_customers']
)
fig_income.update_layout(height=500, title_x=0.5)
fig_income.show()

# 3. Regional analysis
fig_region = px.bar(
    seg_region,
    x='region',
    y='default_rate',
    title='🌍 Default Rate by Region (Interactive)',
    color='default_rate',
    color_continuous_scale='Plasma',
    hover_data=['avg_bill', 'n_bills', 'n_customers']
)
fig_region.update_layout(height=500, title_x=0.5, xaxis_tickangle=-45)
fig_region.show()

# 4. Collections actions pie chart
fig_actions = px.pie(
    actions_dist,
    values='n',
    names='action',
    title='📞 Collections Actions Distribution (Interactive)',
    hover_data=['unique_customers', 'percentage']
)
fig_actions.update_layout(height=500, title_x=0.5)
fig_actions.show()

# 5. Scatter plot analysis
fig_scatter = px.scatter(
    kpi_monthly,
    x='billed',
    y='default_rate',
    size='bill_count',
    hover_data=['month', 'paid'],
    title='💡 Billed Amount vs Default Rate (Interactive)',
    labels={'billed': 'Total Billed Amount (£)', 'default_rate': 'Default Rate'}
)
fig_scatter.update_layout(height=500, title_x=0.5)
fig_scatter.show()

print("\n✨ Interactive Features:")
print("   • Hover over data points for detailed information")
print("   • Zoom using mouse wheel or zoom controls")
print("   • Pan by clicking and dragging")
print("   • Double-click to reset zoom")
print("   • Use legend to show/hide data series")

## 7) Static Charts for Documentation

In [ ]:
# Set up plotting style for static exports
sns.set_theme(style="whitegrid")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 200

# Ensure output directory exists
os.makedirs('../outputs/figures', exist_ok=True)

print("Creating static charts for README documentation...")

# 1) Monthly default rate trend
plt.figure(figsize=(12, 6))
plt.plot(kpi_monthly['month'], kpi_monthly['default_rate'], 
         marker='o', linewidth=2, markersize=6)
plt.title('Monthly Default Rate Trend', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Default Rate', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/figures/monthly_default_rate.png', dpi=200, bbox_inches='tight')
plt.show()
print("✓ Saved monthly_default_rate.png")

# 2) Default rate by income band
plt.figure(figsize=(10, 6))
sns.barplot(data=seg_income, x='income_band', y='default_rate', hue='income_band', legend=False)
plt.title('Default Rate by Income Band', fontsize=16, fontweight='bold')
plt.xlabel('Income Band', fontsize=12)
plt.ylabel('Default Rate', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../outputs/figures/default_by_income.png', dpi=200, bbox_inches='tight')
plt.show()
print("✓ Saved default_by_income.png")

# 3) Default rate by region
plt.figure(figsize=(12, 6))
sns.barplot(data=seg_region, x='region', y='default_rate', hue='region', legend=False)
plt.title('Default Rate by Region', fontsize=16, fontweight='bold')
plt.xlabel('Region', fontsize=12)
plt.ylabel('Default Rate', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../outputs/figures/default_by_region.png', dpi=200, bbox_inches='tight')
plt.show()
print("✓ Saved default_by_region.png")

# 4) Collections actions distribution
plt.figure(figsize=(10, 6))
sns.barplot(data=actions_dist, x='action', y='n', hue='action', legend=False)
plt.title('Collections Actions Volume', fontsize=16, fontweight='bold')
plt.xlabel('Action Type', fontsize=12)
plt.ylabel('Number of Actions', fontsize=12)
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig('../outputs/figures/actions_distribution.png', dpi=200, bbox_inches='tight')
plt.show()
print("✓ Saved actions_distribution.png")

## 8) Actionable Insights

In [ ]:
# Key insights summary
print("=== KEY INSIGHTS ===")
print()

# Overall portfolio performance
overall_default = kpi_monthly['default_rate'].mean()
print(f"📊 Overall Portfolio Default Rate: {overall_default:.1%}")

# Trend analysis
recent_trend = kpi_monthly.tail(6)['default_rate'].mean()
early_trend = kpi_monthly.head(6)['default_rate'].mean()
trend_change = recent_trend - early_trend
print(f"📈 Recent 6-month trend: {trend_change:+.1%} vs early period")

# Highest risk segments
highest_risk_income = seg_income.iloc[0]
lowest_risk_income = seg_income.iloc[-1]
print(f"🎯 Highest risk income band: {highest_risk_income['income_band']} ({highest_risk_income['default_rate']:.1%})")
print(f"🎯 Lowest risk income band: {lowest_risk_income['income_band']} ({lowest_risk_income['default_rate']:.1%})")

highest_risk_region = seg_region.iloc[0]
print(f"🌍 Highest risk region: {highest_risk_region['region']} ({highest_risk_region['default_rate']:.1%})")

# Collections activity
total_actions = actions_dist['n'].sum()
most_common_action = actions_dist.iloc[0]
print(f"📞 Total collections actions: {total_actions:,}")
print(f"📞 Most common action: {most_common_action['action']} ({most_common_action['n']:,} times)")

# Vulnerability insights
vulnerable_rate = vulnerability_analysis[vulnerability_analysis['vulnerability_status'] == 'Vulnerable']['default_rate'].iloc[0]
non_vulnerable_rate = vulnerability_analysis[vulnerability_analysis['vulnerability_status'] == 'Not Vulnerable']['default_rate'].iloc[0]
print(f"🛡️ Vulnerable customers default rate: {vulnerable_rate:.1%}")
print(f"🛡️ Non-vulnerable customers default rate: {non_vulnerable_rate:.1%}")

In [ ]:
# Display comprehensive data tables
print("\n=== COMPREHENSIVE DATA TABLES ===")

print("\n📅 MONTHLY PERFORMANCE (All Data):")
display(kpi_monthly)

print("\n💰 INCOME BAND ANALYSIS:")
display(seg_income)

print("\n🌍 REGIONAL ANALYSIS:")
display(seg_region)

print("\n📞 COLLECTIONS ACTIONS:")
display(actions_dist)

print("\n⚡ TARIFF ANALYSIS:")
display(tariff_analysis)

print("\n🛡️ VULNERABILITY ANALYSIS:")
display(vulnerability_analysis)

## 9) Conclusions & Next Steps

### Key Findings:

1. **Portfolio Performance**: The overall default rate provides a baseline for collections effectiveness
2. **Demographic Risk**: Clear variation in default rates across income bands and regions
3. **Collections Activity**: Understanding of current intervention patterns and volumes
4. **Seasonal Patterns**: Monthly trends reveal potential seasonal effects on payment behaviour
5. **Vulnerability Impact**: Vulnerable customers show different payment patterns

### Recommended Actions:

1. **Targeted Interventions**: Focus collections efforts on highest-risk segments
2. **Early Warning Systems**: Use demographic and regional data for proactive identification
3. **Process Optimisation**: Analyse collections action effectiveness and timing
4. **Monitoring**: Regular dashboard updates to track performance improvements
5. **Vulnerable Customer Support**: Special handling for vulnerable customers

### Potential Extensions:

- **Cohort Analysis**: Track customer payment behaviour over time
- **Tariff Analysis**: Examine default rates by tariff type
- **Arrears Balances**: Calculate outstanding amounts in addition to default rates
- **Predictive Modelling**: Build early warning models for proactive collections
- **ROI Analysis**: Measure collections action effectiveness and cost-benefit

In [ ]:
# Close database connection
con.close()
print("Database connection closed.")
print("\n🎉 Portfolio Collections Dashboard analysis complete!")
print("\n📊 Summary of deliverables:")
print("   ✓ Interactive dashboard with working widgets")
print("   ✓ Enhanced interactive charts with hover details")
print("   ✓ Comprehensive data tables displayed")
print("   ✓ Static charts saved for README")
print("   ✓ Actionable insights and recommendations")